In [ ]:
!pip install dash-dangerously-set-inner-html


In [ ]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import dash
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import geopandas as gpd
from shapely import wkt
import folium
from folium import LayerControl, GeoJson, GeoJsonPopup, GeoJsonTooltip
import branca
import io
import base64
from dash_dangerously_set_inner_html import DangerouslySetInnerHTML


# Read the HTML file content
with open('home.html', 'r') as file:
    home_page_content = file.read()



# # Function to fetch home page HTML from Flask endpoint
# def get_home_page_content():
#     response = requests.get('http://localhost:5000/home-layout')
#     if response.status_code == 200:
#         return DangerouslySetInnerHTML(response.text)
#     else:
#         return html.Div([
#             html.H1("Error loading home page content"),
#             html.P("Unable to fetch data from server.")
#         ])

# Fetch data from the Flask API ER
def fetch_data_er():
    response = requests.get('http://127.0.0.1:5000/landslides/surface')
    return pd.DataFrame(response.json())

def fetch_population_data():
    response = requests.get('http://127.0.0.1:5000/landslides/population')
    return pd.DataFrame(response.json())

def fetch_map():
    response = requests.get('http://127.0.0.1:5000/landslides/map')
    return pd.DataFrame(response.json())

# Function to create folium map and return it as an HTML iframe
def create_folium_map(data, data_type):
    m = folium.Map(location=[45.6101948758674, 9.481178872400372], zoom_start=8)

    # Create a GeoDataFrame from the geometries
    geometries = data['geometry'].tolist()
    gdf = gpd.GeoDataFrame(data, geometry=[wkt.loads(geom) for geom in geometries])

    # Ensure the geometries are in the correct CRS (EPSG:4326)
    if gdf.crs is None:
        gdf.set_crs(epsg=32632, inplace=True)  # Set the original CRS
    gdf.to_crs(epsg=4326, inplace=True)  # Convert to EPSG:4326

    # Add each polygon to the folium map with popups
    for _, row in gdf.iterrows():
        if data_type == 'surface':
            popup_html = f"""
                <strong>Province:</strong> {row['nome']}<br>
                <strong>Area (km²):</strong> {row['ar_kmq']}<br>
                <strong>Moderate LS risk (km²):</strong> {row['ar_fr_p1']}<br>
                <strong>Medium LS risk (km²):</strong> {row['ar_fr_p2']}<br>
                <strong>High LS risk (km²):</strong> {row['ar_fr_p3p4']}<br>
                <strong>Very high LS risk (km²):</strong> {row['ar_fr_p4']}<br>
                <strong>Very high and high LS risk (km²):</strong> {row['ar_fr_p3p4']}<br>
            """
        elif data_type == 'population':
            popup_html = f"""
                <strong>Province:</strong> {row['nome']}<br>
                <strong>Area (km²):</strong> {row['ar_kmq']}<br>
                <strong>Moderate LS risk (no. of inhabitants):</strong> {row['pop_fr_p1']}<br>
                <strong>Medium LS risk (no. of inhabitants):</strong> {row['pop_fr_p2']}<br>
                <strong>High LS risk (no. of inhabitants):</strong> {row['pop_fr_p3']}<br>
                <strong>Very high LS risk (no. of inhabitants):</strong> {row['pop_fr_p4']}<br>
            """
        folium.GeoJson(
            data=row['geometry'],
            name=row['nome'],
            style_function=lambda feature: {
                'fillColor': 'green',  # Setting the fill color to green
                'color': 'black',
                'weight': 2,
                'dashArray': '5, 5'
            },
            tooltip=row['nome'],
            popup=folium.Popup(html=popup_html, max_width=300)
        ).add_to(m)

    map_html = io.BytesIO()
    m.save(map_html, close_file=False)
    map_html.seek(0)
    map_base64 = base64.b64encode(map_html.read()).decode()
    return f'data:text/html;base64,{map_base64}'

# Fetch data from the Flask API UM
def fetch_data_um():
    try:
        response = requests.get('http://127.0.0.1:5000/map')
        response.raise_for_status()
        data = response.json()
        return pd.DataFrame(data)
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return pd.DataFrame()

# Function to determine popup content based on risk type and level
def get_popup_content(risk_type_um, risk_level, selected_column):
    if risk_type_um == 'landslide_surface_area':
        label = 'Landslide Surface Area'
        unit = ' (km²)'
    elif risk_type_um == 'population':
        label = 'Population'
        unit = ' (Number)'
    elif risk_type_um == 'buildings':
        label = 'Buildings'
        unit = ' (Number)'
    else:
        label = 'Data'
        unit = ''

    if risk_level == 'very_high':
        level = 'very high'
    elif risk_level == 'high':
        level = 'high'
    elif risk_level == 'medium':
        level = 'medium'
    else:
        level = 'Risk Level'

    return f"{label} at {level} risk{unit}"

# Define external stylesheets
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# Initialize the Dash app with suppress_callback_exceptions=True
app = Dash(__name__, suppress_callback_exceptions=True)

app.index_string = '''
<!DOCTYPE html>
<html>
    <head>
        <title>AUPE RiskMonitor Dashboard</title>
        <link rel="stylesheet" type="text/css" href="/assets/styles.css">
        {%metas%}
        {%favicon%}
        {%css%}
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
            {%renderer%}
        </footer>
    </body>
</html>
'''

# Define base tile layers with attribution and graceful names
base_tiles = {
    'OpenStreetMap': {
        'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        'attr': 'Map data &copy; OpenStreetMap contributors',
        'name': 'OpenStreetMap'
    },
    'Esri Topo': {
        'url': 'https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png',
        'attr': 'Map data &copy; OpenTopoMap contributors',
        'name': 'Esri Topo'
    },
    'CartoDB Voyager': {
        'url': 'https://{s}.basemaps.cartocdn.com/rastertiles/voyager/{z}/{x}/{y}{r}.png',
        'attr': 'Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
        'name': 'CartoDB Voyager'
    },
    'CartoDB Dark': {
        'url': 'https://{s}.basemaps.cartocdn.com/dark_all/{z}/{x}/{y}{r}.png',
        'attr': 'Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.',
        'name': 'CartoDB Dark'
    }
}


app.layout = html.Div([
    html.Div([
        html.A(
            html.Img(src='/assets/logo2.jpg', style={'height': '60px', 'margin': 'auto', 'display': 'block'}),
        )
    ], style={'textAlign': 'center', 'padding': '20px'}),  # Center the logo and add padding

    dcc.Tabs(id='tabs', value='tab-home', children=[
        dcc.Tab(label='Home', value='tab-home'),
        dcc.Tab(label='Statistics', value='tab-statistics'),
        dcc.Tab(label='Map', value='tab-map', children=[
            dcc.Tabs(id='map-tabs', value='map-tab-regional', children=[
                dcc.Tab(label='Regional Analysis', value='map-tab-regional', children=[
                    html.Div([
                        html.Div([
                            html.Label('Risk Type:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                            dcc.Dropdown(
                                id='risk-type-dropdown_um',
                                options=[
                                    {'label': 'Landslide Surface Area', 'value': 'landslide_surface_area'},
                                    {'label': 'Population', 'value': 'population'},
                                    {'label': 'Buildings', 'value': 'buildings'}
                                ],
                                value='landslide_surface_area',
                                clearable=False,
                                style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                                       'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                            ),
                        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),

                        html.Div([
                            html.Label('Risk Level:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                            dcc.Dropdown(
                                id='risk-level-dropdown_um',
                                options=[
                                    {'label': 'Very High', 'value': 'very_high'},
                                    {'label': 'High', 'value': 'high'},
                                    {'label': 'Medium', 'value': 'medium'}
                                ],
                                value='very_high',
                                clearable=False,
                                style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                                       'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                            ),
                        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),

                        html.Div([
                            html.Label('Base Tile Layer:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                            dcc.Dropdown(
                                id='base-tile-dropdown',
                                options=[{'label': tile_name, 'value': tile_info['url']} for tile_name, tile_info in base_tiles.items()],
                                value=base_tiles['OpenStreetMap']['url'],
                                clearable=False,
                                style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                                       'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                            ),
                        ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'})
                    ], style={'textAlign': 'center'}),

                    html.Div([
                        html.Div([
                            html.Label('Opacity:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                            dcc.Slider(
                                id='opacity-slider',
                                min=0,
                                max=1,
                                step=0.01,
                                value=0.7,
                                marks={0: '0', 1: '1'},
                                tooltip={'always_visible': True, 'placement': 'bottom'}
                            ),
                            html.Div(id='opacity-output-container', style={'marginTop': '10px'})
                        ], style={'width': '100%', 'display': 'inline-block', 'verticalAlign': 'top', 'padding': '10px', 'textAlign': 'center',}),
                    ], style={'width': '30%', 'margin': '0 auto'}),

                    
                    html.Div([
                        html.Div(id='map-container', children=[
                            html.Iframe(id='map', srcDoc=None, width='100%', height='100%')
                        ], style={'width': '80%', 'margin': '0 auto', 'height': '80vh'})
                    ]),

                    html.Div(
                        children=[
                            html.H2("Graphical Overview", style={'textAlign': 'center', 'color': '#000'}),
                            html.P("Thw bar graph below gives the overall overview of the above selected data.", style={'textAlign': 'center', 'color': '#000'}),
                        ],
                        style={
                            'width': '100%',  # Full width of the parent container
                            'maxWidth': '600px',  # Maximum width
                            'height': 'auto',  # Height based on content
                            'border': '2px solid #333',  # Border of the rectangle
                            'backgroundColor': '#fff',  # Background color of the rectangle
                            'borderRadius': '5px',  # Rounded corners
                            'padding': '20px',  # Space inside the border
                            'margin': '20px auto',  # Center the rectangle on the page
                            'boxShadow': '0px 4px 8px rgba(0,0,0,0.3)',  # Shadow effect
                            'textAlign': 'center',  # Center-align text
                            'color': '#fff',  # Text color
                        }
                    ),



                    html.Div([
                        html.Div([
                            html.Label('Sort Order:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                            dcc.Dropdown(
                                id='sort-order-dropdown',
                                options=[
                                    {'label': 'Default Order', 'value': 'initial'},
                                    {'label': 'Highest to Lowest', 'value': 'desc'},
                                    {'label': 'Lowest to Highest', 'value': 'asc'}
                                ],
                                value='initial',
                                clearable=False,
                                style={'width': '100%', 'fontSize': '14px', 'padding': '8px',  'marginRight': '10px',
                                       'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                            )
                        ], style={'width': '100%', 'padding': '10px'}),

                    ], style={'width': '30%', 'margin': '0 auto'}),
                        
                    html.Div([
                        dcc.Graph(id='bar-chart')
                    ], style={'width': '80%', 'margin': '0 auto'}),
                ]),

                
                 dcc.Tab(label='Provincial Analysis', value='map-tab-provincial', children=[                     
                     html.Div([
                         html.Div([
                            html.Div([
                                html.Label('Risk Type:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                                dcc.Dropdown(
                                    id='data-type-dropdown-map',
                                    options=[
                                    {'label': 'Landslide Surface Area', 'value': 'surface'},
                                    {'label': 'Population', 'value': 'population'}
                                    ],
                                    placeholder='Select data type',
                                    clearable=False,
                                    style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                                           'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                                ),
                            ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),
    
                            html.Div([
                                html.Label('Province:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                                dcc.Dropdown(
                                    id='province-dropdown-map',
                                    options=[],
                                    multi=True,  # Allow multiple selections
                                    placeholder='Select provinces to view on the map',
                                    clearable=False,
                                    style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                                           'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                                ),
                            ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),

                     ], style = {'textAlign': 'center'}),
                        # # Div to hold the map
                        html.Div([
                        html.Div(id='map-container-er', children=[
                            html.Iframe(id='map-provinces', srcDoc=None, width='100%', height='auto')
                            ], style={'width': '80%', 'margin': '0 auto'})
                        ])
                    ])
                ])
            ])
        ])
    ]),
    # Tab content
    html.Div(id='tabs-content')
], style={'fontFamily': 'Arial, sans-serif', 'margin': 'auto', 'padding': '0', 'backgroundColor': '#f2f7e6', 'width': '100%', 'justifyContent': 'center'})

# Callback to update tab content
@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def render_content(tab):
    if tab == 'tab-home':
        return DangerouslySetInnerHTML(home_page_content)
        # return get_home_page_content()
    if tab == 'tab-statistics':
        return html.Div([
            # Dropdown to select data type
            html.Div([
                html.Label('Risk Type:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                dcc.Dropdown(
                    id='data-type-dropdown-stats',
                    options=[
                    {'label': 'Landslide Surface Area', 'value': 'surface'},
                    {'label': 'Population', 'value': 'population'}
                    ],
                    placeholder='Select data type',
                    clearable=False,
                    style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                           'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                ),
            ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),


            # Dropdown to select provinces for statistics
            html.Div([
                html.Label('Province:', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                dcc.Dropdown(
                    id='province-dropdown-stats',
                    options=[],
                    multi=True,  # Allow multiple selections
                    placeholder='Select provinces to view on the map',
                    clearable=False,
                    style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                           'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                ),
            ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),

            # Dropdown to select landslide risk Level (previuosly it was risk type. I only changed it here)
            html.Div([
                html.Label('Risk Level', style={'fontWeight': 'bold', 'marginRight': '10px'}),
                dcc.Dropdown(
                    id='risk-type-dropdown',
                    options=[], # Options will be updated dynamically
                    placeholder='Select landslide risk Level',
                    clearable=False,
                    style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                           'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                ),
            ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),


            # Dropdown for choosing chart or table
            html.Div([
                html.Label('Select Visualization:', style={'fontWeight': 'bold', 'marginBottom': '10px'}),  # Style for the label
                dcc.Dropdown(
                id='visualization-checkbox',
                options=[
                    {'label': 'Bar Chart', 'value': 'bar'},
                    {'label': 'Pie Chart', 'value': 'pie'},
                    {'label': 'Table', 'value': 'table'}
                    ],
                    placeholder='Select the Visualization',
                    multi=True,  # Allow multiple selections
                    clearable=False,
                    value=[],  # No default selection
                    style={'width': '100%', 'fontSize': '14px', 'padding': '8px', 'marginRight': '10px',
                           'border': '1px solid #ccc', 'borderRadius': '4px', 'color': '#333'}
                ),
                
            ], style={'display': 'inline-block', 'width': '30%', 'verticalAlign': 'top', 'padding': '10px'}),
            html.Div(id='visualizations-container',
                ),
        ], style={'width':'80%', 'margin':'0 auto', 'textAlign': 'center'})
# Callback to update dropdown options for statistics
@app.callback(
    Output('province-dropdown-stats', 'options'),
    Input('data-type-dropdown-stats', 'value')
)
def update_province_dropdown_stats(data_type):
    if data_type == 'surface':
        df = fetch_data_er()
    elif data_type == 'population':
        df = fetch_population_data()
    else:
        return []
    
    options = [{'label': name, 'value': name} for name in np.sort(df['nome'].unique())]
    return options

# Callback to update dropdown options for map section based on selected data type
@app.callback(
    Output('province-dropdown-map', 'options'),
    Input('data-type-dropdown-map', 'value')
)

def update_map_province_dropdown(data_type):
    if data_type == 'surface':
        df = fetch_data_er()
    elif data_type == 'population':
        df = fetch_population_data()
    else:
        return []
    
    options = [{'label': name, 'value': name} for name in np.sort(df['nome'].unique())]
    return options

# Callback to update risk type dropdown options based on selected data type
@app.callback(
    Output('risk-type-dropdown', 'options'),
    Input('data-type-dropdown-stats', 'value')
)
def update_risk_type_dropdown(data_type):
    if data_type == 'surface':
        options = [
            {'label': 'Low LS risk (km²)', 'value': 'ar_fr_p1'},
            {'label': 'Medium LS risk (km²)', 'value': 'ar_fr_p2'},
            {'label': 'High LS risk (km²)', 'value': 'ar_fr_p3'},
            {'label': 'Very high LS risk (km²)', 'value': 'ar_fr_p4'},
            {'label': 'Very high and high LS risk (km²)', 'value': 'ar_fr_p3p4'},
            {'label': 'Moderate LS risk (%)', 'value': 'ar_frp1_p'},
            {'label': 'Medium LS risk (%)', 'value': 'ar_frp2_p'},
            {'label': 'High LS risk (%)', 'value': 'ar_frp3_p'},
            {'label': 'Very high LS risk (%)', 'value': 'ar_frp4_p'},
            {'label': 'Very high and high LS risk (%)', 'value': 'ar_frp3p4p'}
        ]
    elif data_type == 'population':
        options = [
            {'label': 'Moderate LS risk (no. of inhabitants)', 'value': 'pop_fr_p1'},
            {'label': 'Medium LS risk (no. of inhabitants)', 'value': 'pop_fr_p2'},
            {'label': 'High LS risk (no. of inhabitants)', 'value': 'pop_fr_p3'},
            {'label': 'Very high LS risk (no. of inhabitants)', 'value': 'pop_fr_p4'},
            {'label': 'Very high and high LS risk (no. of inhabitants)', 'value': 'popfr_p3p4'},
            {'label': 'Moderate LS risk (%)', 'value': 'popfrp1_p'},
            {'label': 'Medium LS risk (%)', 'value': 'popfrp2_p'},
            {'label': 'High LS risk (%)', 'value': 'popfrp3_p'},
            {'label': 'Very high LS risk (%)', 'value': 'popfrp4_p'},
            {'label': 'Very high and high LS risk (%)', 'value': 'popfrp3p4p'}
        ]
    else:
        options = []
    return options

# Callback to update map content based on selected provinces and data type
@app.callback(
    Output('map-container-er', 'children'),
    Input('province-dropdown-map', 'value'),
    Input('data-type-dropdown-map', 'value')
)
def update_map_er(selected_provinces, data_type):
    if not selected_provinces:
        return html.Div(["No map data available for selected provinces."], style= {'textAlign': 'center' })

    df = fetch_map()
    filtered_df = df[df['nome'].isin(selected_provinces)]

    if filtered_df.empty:
        return html.Div(["No map data available for selected provinces."], style= {'textAlign': 'center' })

    map_html = create_folium_map(filtered_df, data_type)
    return html.Iframe(src=map_html, width='100%', height='600')

# Callback to update bar chart, pie chart, or table based on selected options
@app.callback(
    Output('visualizations-container', 'children'),
    Input('province-dropdown-stats', 'value'),
    Input('risk-type-dropdown', 'value'),
    Input('visualization-checkbox', 'value'),
    Input('data-type-dropdown-stats', 'value')
)
def update_visualizations(selected_provinces, risk_type, visualizations, data_type):
    if not selected_provinces or not risk_type or not visualizations or not data_type:
        return html.Div(["Please choose one of the visualization options."], style= {'textAlign': 'center' })

    if data_type == 'surface':
        df = fetch_data_er()
    elif data_type == 'population':
        df = fetch_population_data()
    else:
        return html.Div(["Invalid data type selected."], style= {'textAlign': 'center' })

    filtered_df = df[df['nome'].isin(selected_provinces)]

    if filtered_df.empty:
        return html.Div(["No data available for selected provinces."], style= {'textAlign': 'center' })

    outputs = []

    # Map of risk type value to label
    risk_type_map = {
        'ar_fr_p1': 'Low LS risk (km²)',
        'ar_fr_p2': 'Medium LS risk (km²)',
        'ar_fr_p3': 'High LS risk (km²)',
        'ar_fr_p4': 'Very high LS risk (km²)',
        'ar_fr_p3p4': 'Very high and high LS risk (km²)',
        'ar_frp1_p': 'Moderate LS risk (%)',
        'ar_frp2_p': 'Medium LS risk (%)',
        'ar_frp3_p': 'High LS risk (%)',
        'ar_frp4_p': 'Very high LS risk (%)',
        'ar_frp3p4p': 'Very high and high LS risk (%)',
        'pop_fr_p1': 'Moderate LS risk (no. of inhabitants)',
        'pop_fr_p2': 'Medium LS risk (no. of inhabitants)',
        'pop_fr_p3': 'High LS risk (no. of inhabitants)',
        'pop_fr_p4': 'Very high LS risk (no. of inhabitants)',
        'popfr_p3p4': 'Very high and high LS risk (no. of inhabitants)',
        'popfrp1_p': 'Moderate LS risk (%)',
        'popfrp2_p': 'Medium LS risk (%)',
        'popfrp3_p': 'High LS risk (%)',
        'popfrp4_p': 'Very high LS risk (%)',
        'popfrp3p4p': 'Very high and high LS risk (%)'
    }

    risk_type_label = risk_type_map.get(risk_type, risk_type)

    if 'bar' in visualizations:
        fig_bar = px.bar(
            filtered_df,
            x='nome',
            y=risk_type,
            title=f'{risk_type_label} for Selected Provinces',
            labels={'nome': 'Province', risk_type: risk_type_label}
        )
        outputs.append(dcc.Graph(figure=fig_bar))

    if 'pie' in visualizations:
        pie_data = []
        total_percentage = filtered_df[risk_type].sum()

        for province in selected_provinces:
            province_data = filtered_df[filtered_df['nome'] == province]
            if not province_data.empty:
                province_percentage = province_data[risk_type].values[0]
                pie_data.append({
                    'province': province,
                    'percentage': province_percentage
                })

        remaining_percentage = 100.0 - total_percentage
        if remaining_percentage > 0:
            pie_data.append({
                'province': 'Remaining',
                'percentage': remaining_percentage
            })

        pie_df = pd.DataFrame(pie_data)

        fig_pie = px.pie(
            pie_df,
            values='percentage',
            names='province',
            title=f'The percentage of {risk_type_label} in Selected Provinces',
            color_discrete_sequence=px.colors.qualitative.Pastel
        )
        outputs.append(dcc.Graph(figure=fig_pie))

    if 'table' in visualizations:
        if data_type == 'surface':
            columns = ['nome','ar_kmq', 'ar_fr_p1', 'ar_fr_p2', 'ar_fr_p3', 'ar_fr_p4']
            column_names = ['Province','Area (km²)', 'Moderate LS risk (km²)', 'Medium LS risk (km²)', 'High LS risk (km²)', 'Very High LS risk (km²)']
        elif data_type == 'population':
            columns = ['nome','ar_kmq', 'pop_fr_p1', 'pop_fr_p2', 'pop_fr_p3','pop_fr_p4']
            column_names = ['Province','Area (km²)', 'Moderate LS risk (no. of inhabitants)', 'Medium LS risk (no. of inhabitants)', 'High LS risk (no. of inhabitants)','Very High LS risk (no. of inhabitants)']
        else:
            return html.Div("Invalid data type selected.")

        table = dash_table.DataTable(
            data=filtered_df[columns].to_dict('records'),
            columns=[{"name": col_name, "id": col_id} for col_name, col_id in zip(column_names, columns)],
            style_cell={
                'textAlign': 'left',
                'padding': '10px',
                'whiteSpace': 'normal',
                'height': 'auto',
                'border': '1px solid grey'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold',
                'border': '1px solid black'
            },
            style_data={
                'border': '1px solid grey',
                'backgroundColor': 'rgb(248, 248, 248)'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(230, 230, 230)',
                },
            ],
            style_table={
                'maxHeight': '500px',
                'overflowY': 'scroll',
                'borderCollapse': 'collapse'
            },
        )
        outputs.append(table)
        
        # tables_km = outputs.append(table)
        # Second table for percentage-based risk data
        if data_type == 'surface':
            columns_percentage = ['nome','ar_kmq', 'ar_frp1_p', 'ar_frp2_p', 'ar_frp3_p', 'ar_frp4_p']
            column_names_percentage = ['Province','Area (km²)', 'Moderate LS risk (%)', 'Medium LS risk (%)', 'High LS risk (%)', 'Very High LS risk (%)']
        elif data_type == 'population':
            columns_percentage = ['nome','ar_kmq', 'popfrp1_p', 'popfrp2_p', 'popfrp3_p','popfrp4_p']
            column_names_percentage = ['Province','Area (km²)', 'Moderate LS risk (%)', 'Medium LS risk (%)', 'High LS risk (%)','Very High LS risk (%)']
        else:
            return html.Div("Invalid data type selected.")

        table_percentage = dash_table.DataTable(
            data=filtered_df[columns_percentage].to_dict('records'),
            columns=[{"name": col_name, "id": col_id} for col_name, col_id in zip(column_names_percentage, columns_percentage)],
            style_cell={
                'textAlign': 'left',
                'padding': '10px',
                'whiteSpace': 'normal',
                'height': 'auto',
                'border': '1px solid grey'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold',
                'border': '1px solid black'
            },
            style_data={
                'border': '1px solid grey',
                'backgroundColor': 'rgb(248, 248, 248)'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(230, 230, 230)',
                },
            ],
            style_table={
                'maxHeight': '500px',
                'overflowY': 'scroll',
                'borderCollapse': 'collapse'
            },
        )
        outputs.append(table_percentage)
        # tables_percent = outputs.append(table_percentage)
        # return html.Div(tables_percent)

    return html.Div(outputs)


# Callback to update the map and opacity
@app.callback(
    Output('map', 'srcDoc'),
    [Input('risk-type-dropdown_um', 'value'),
     Input('risk-level-dropdown_um', 'value'),
     Input('opacity-slider', 'value'),
     Input('base-tile-dropdown', 'value')]
)
def update_map_um(risk_type_um, risk_level, opacity, base_tile_url):
    df = fetch_data_um()

    if df.empty:
        return None  # Return None if data fetching failed

    # Convert WKT to GeoDataFrame
    df['geometry'] = df['geom_wkt'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Set the initial CRS to EPSG:32632 and then transform to EPSG:4326
    gdf.set_crs('EPSG:32632', inplace=True)
    gdf = gdf.to_crs(epsg=4326)

    # Drop the 'geom_wkt' column
    gdf = gdf.drop(columns=['geom_wkt'])

    # Map risk type and level to the corresponding column
    risk_columns = {
        'landslide_surface_area': {
            'very_high': 'ar_fr_p4',
            'high': 'ar_fr_p3',
            'medium': 'ar_fr_p2'
        },
        'population': {
            'very_high': 'pop_fr_p4',
            'high': 'pop_fr_p3',
            'medium': 'pop_fr_p2'
        },
        'buildings': {
            'very_high': 'ed_fr_p4',
            'high': 'ed_fr_p3',
            'medium': 'ed_fr_p2'
        }
    }

    selected_column = risk_columns[risk_type_um][risk_level]

    # Define custom colormaps for each risk type using ColorBrewer palettes
    colormaps = {
        'landslide_surface_area': branca.colormap.LinearColormap(
            colors=['#f7fbff', '#c6dbef', '#6baed6', '#3182bd', '#08519c'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Landslide Surface Area Risk Level (km²) "
        ),
        'population': branca.colormap.LinearColormap(
            colors=['#f7fcf5', '#c7e9c0', '#74c476', '#31a354', '#006d2c'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Population Risk Level (Number)"
        ),
        'buildings': branca.colormap.LinearColormap(
            colors=['#fff5f0', '#fcbba1', '#fc9272', '#fb6a4a', '#cb181d'],
            vmin=gdf[selected_column].min(),
            vmax=gdf[selected_column].max(),
            caption="Buildings Risk Level (Number)"
        )
    }

    colormap = colormaps[risk_type_um]

    # Get the attribution for the selected base tile
    base_tile_info = next(tile_info for tile_info in base_tiles.values() if tile_info['url'] == base_tile_url)
    base_tile_attr = base_tile_info['attr']
    base_tile_name = base_tile_info['name']

    # Create a map
    m = folium.Map(location=[45.05780847800959, 9.635009765625002], zoom_start=8, tiles=None)

    # Add the base tile to the map without adding it to the layer control
    folium.TileLayer(
        tiles=base_tile_url,
        attr=base_tile_attr,
        name=base_tile_name,
        control=True  # Exclude from layer control
    ).add_to(m)
    
    # Add a choropleth layer to the map
    def style_function(feature):
        return {
            'fillColor': colormap(feature['properties'][selected_column]),
            'color': 'black',
            'fillOpacity': opacity,
            'weight': 0.5
        }

    # Create GeoJsonPopup with dynamic content
    popup = GeoJsonPopup(
        fields=['nome', selected_column],
        aliases=['Region', get_popup_content(risk_type_um, risk_level, selected_column)],  # Use function to get dynamic content
        localize=True,
        labels=True,
        style="backgroundColor: yellow;",
    )

    # Create GeoJsonTooltip with dynamic content
    tooltip = GeoJsonTooltip(
        fields=['nome', selected_column, "ar_kmq"],
        aliases=['Region', get_popup_content(risk_type_um, risk_level, selected_column), "Total Area (km²):"],
        localize=True,
        sticky=True,
        style="""
            backgroundColor: #F0EFEF;
            border: 2px solid black;
            borderRadius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )


    
    # Add GeoJson layer with popup and tooltip to the map
    g = GeoJson(
        gdf,
        name='Landslide Risk Choropleth',
        style_function=style_function,
        popup=popup,
        highlight_function=lambda x: {'weight':2, 'fillOpacity': 1.0},
        tooltip=tooltip,
    ).add_to(m)

    # Add layer control and colormap to the map
    LayerControl(position='topleft', collapsed=True,).add_to(m)
    colormap.add_to(m)

    # Return the HTML representation of the map
    return m._repr_html_()

# Update the bar chart callback to include the sorting order

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('risk-type-dropdown_um', 'value'),
     Input('risk-level-dropdown_um', 'value'),
     Input('sort-order-dropdown', 'value')]
)
def update_bar_chart(risk_type_um, risk_level, sort_order):
    df = fetch_data_um()

    if df.empty:
        return {}

    # Map risk type and level to the corresponding column
    risk_columns = {
        'landslide_surface_area': {
            'very_high': 'ar_fr_p4',
            'high': 'ar_fr_p3',
            'medium': 'ar_fr_p2'
        },
        'population': {
            'very_high': 'pop_fr_p4',
            'high': 'pop_fr_p3',
            'medium': 'pop_fr_p2'
        },
        'buildings': {
            'very_high': 'ed_fr_p4',
            'high': 'ed_fr_p3',
            'medium': 'ed_fr_p2'
        }
    }

    selected_column = risk_columns[risk_type_um][risk_level]

    # Sort the DataFrame based on the sort_order
    if sort_order == 'asc':
        df = df.sort_values(by=selected_column, ascending=True)
    elif sort_order == 'desc':
        df = df.sort_values(by=selected_column, ascending=False)

    # Define a color scale for the bar chart based on risk type
    color_scales = {
        'landslide_surface_area': px.colors.sequential.Blues,
        'population': px.colors.sequential.Greens,
        'buildings': px.colors.sequential.Reds
    }

    color_scale = color_scales[risk_type_um]

    # Create a bar chart with a vibrant color scale
    fig = px.bar(df, x='nome', y=selected_column,
                 labels={'nome': 'Region', selected_column: get_popup_content(risk_type_um, risk_level, selected_column)},
                 title=f'{risk_type_um.replace("_", " ").capitalize()} at {risk_level.replace("_", " ").capitalize()} Risk',
                 color=selected_column, color_continuous_scale=color_scale)

    # Update y-axis label to format nicely
    unit_label = ' (km²)' if 'area' in selected_column else ' (Number)'
    y_axis_label = f"{risk_type_um.replace('_', ' ').capitalize()} at {risk_level.replace('_', ' ').capitalize()} Risk{unit_label}"
    fig.update_yaxes(title_text=y_axis_label)

    return fig

# Callback to reset the sort order when the risk type is changed
@app.callback(
    Output('sort-order-dropdown', 'value'),
    [Input('risk-type-dropdown_um', 'value')]
)
def reset_sort_order(_):
    return 'initial'



# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=3000)
